In [17]:
import json
import numpy as np

In [18]:
class Loadfileset():
        def __init__(self, jsonfilename) :
            with open(jsonfilename) as f :
                self.handler = json.load(f)
    
        
        def Show(self , verbosity=1):
            if verbosity==1 :
                for key, value in self.handler.items() :
                    rich.print(key+" : ", list(value.keys()))
            elif verbosity==2 :
                for key, value in self.handler.items() :
                    rich.print(key+" : ", list(value.keys()), "\n")
                    for subkey , subvalue in value.items() :
                        rich.print("\t"+subkey+" : ")
                        for file in subvalue :
                            rich.print("\t", file)
            elif verbosity==3 :
                for key, value in self.handler.items() :
                    rich.print(key+" : ", list(value.keys()), "\n")
                    for subkey , subvalue in value.items() :
                        rich.print("\t"+subkey+" : ")
                        for subsubkey, subsubvalue in subvalue.items() :
                            try :
                                for file in subvalue :
                                    rich.print("\t", file)
                            except:
                                rich.print("\t"+subsubkey+" : ")
        
        def getFileset(self, mode ,superkey, key, redirector ) :
            if redirector=="fnal":
                redirector_string = "root://cmsxrootd.fnal.gov//"
            elif redirector=="infn":
                redirector_string = "root://xrootd-cms.infn.it//"
            elif redirector=="wisc":
                redirector_string = "root://pubxrootd.hep.wisc.edu//"
            elif redirector=="unl":
                redirector_string = "root://xrootd-local.unl.edu:1094//"
            elif redirector=="kisti":
                redirector_string = "root://cms-xrdr.sdfarm.kr:1094//xrd//"
            raw_fileset = self.handler[mode][superkey][key] 
            requested_fileset = {superkey : [redirector_string+filename for filename in raw_fileset]}
            return requested_fileset
        
        def getraw(self):
            #load the raw dictionary
            full_fileset = self.handler
            return full_fileset
def getDataset(keymap, load=True, dict = None, files=None, begin=0, end=0, mode = "sequential"):
    #Warning : Never use 'files' with 'begin' and 'end'
    fileset = Loadfileset("samples/2018_samples.json")
    fileset_dict = fileset.getraw()
    MCmaps =[
        "DataA",
        "DataB",
        "DataC",
        "DataD",
        "MCTTbar1l1v",
        "MCTTbar0l0v",
        "MCTTbar2l2v",
        "MCZ1Jets",
        "MCZ2Jets",
        "MCWlvJets",
        "MCDYJets",
        "MCDYJets_M4to50",
        "MCSingleTop1",
        "MCSingleTop2",
        "MCVV",
        "MCHiggs",
        "MCQCD",
        "MCDYincl",
        "MCDYJetsZpT1",
        "MCDYJetsZpT2",
        "MCDYJetsZpT3"
    ]
    
    runnerfileset = buildFileset(fileset_dict[keymap],"fnal")
    flat_list={}
    flat_list[keymap] = []

    if mode == "sequential":
        if end - begin < 0:
            print("Invalid begin and end values.\nFalling back to full dataset...")
            outputfileset = runnerfileset
        else:
            #indexer
            index={}
            i = 1
            for key in runnerfileset.keys() :
                index[key] = []
                for file in runnerfileset[key] :
                    index[key].append(i)
                    i += 1

            accept = np.arange(begin,end+1,1)
            print(accept)
            temp = {}
            for key in runnerfileset.keys() :
                temp[key] = []
                for i in range(len(runnerfileset[key])) :
                    if index[key][i] in accept :
                        temp[key].append(runnerfileset[key][i])

            outputfileset = temp
    elif mode == "divide" :
        if files == None:
            print("Invalid number of files.\nFalling back to full dataset...")
            outputfileset = runnerfileset
        else:
            # Divide the share of files from all the 8 categories of ZJets_NuNu
            file_number = 0
            while file_number < files :
                for key in runnerfileset.keys():
                    if file_number >= files :
                        break
                    flat_list[keymap] += [runnerfileset[key][0]]
                    runnerfileset[key] = runnerfileset[key][1:]
                    file_number += 1
            outputfileset = {keymap : flat_list[keymap]}
    else:
        print("Invalid mode of operation", mode)
        raise KeyError
    
    print("Running ", np.array([len(value) for value in outputfileset.values()]).sum(), " files...")
    return outputfileset
def buildFileset(dict , redirector):
    '''
    To return a run-able dict with the appropriate redirector.
    Please input a dictionary which is only singly-nested
    '''
    redirectors = {
        "fnal": "root://cmsxrootd.fnal.gov//",
        "infn": "root://xrootd-cms.infn.it//",
        "wisc": "root://pubxrootd.hep.wisc.edu//",
        "unl":  "root://xrootd-local.unl.edu:1094/",
        "kisti": "root://cms-xrdr.sdfarm.kr:1094//xrd/",
        "hdfs": "/hdfs",
        "commonfs": "/commonfs"

    }

    if (redirector=="fnal") | (redirector==1) :
        redirector_string = redirectors["fnal"]
    elif (redirector=="infn") | (redirector==2) :
        redirector_string = redirectors["infn"]
    elif (redirector=="wisc") | (redirector==3):
        redirector_string = redirectors["wisc"]
    elif (redirector=="unl") | (redirector==4):
        redirector_string = redirectors["unl"]
    elif (redirector=="kisti") | (redirector==5):
        redirector_string = redirectors["kisti"]
    elif (redirector=="hdfs") | (redirector==6):
        redirector_string = redirectors["hdfs"]
    elif (redirector=="commonfs") | (redirector==7):
        redirector_string = redirectors["commonfs"]

    temp = dict 
    output = {}
    for key in temp.keys() :
        try :
            g = temp[key]
            if isinstance(g,list):
                templist = []
                for filename in g :
                    filename = filename[filename.find("/store/") :]
                    templist.append(redirector_string+filename)
                output[key] = templist
        except :
            raise KeyError
    return output

In [23]:
with open("samples/2018_samples.json") as f: #load the fileset
            filedict = json.load(f)
files = getDataset(
            keymap="MCZ2Jets",
            load=False ,
            dict=filedict,
            mode="sequential",
            begin=1,
            end=200
            )

[  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108
 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126
 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144
 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162
 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180
 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198
 199 200]
Running  91  files...


In [25]:
files.keys()

dict_keys(['Z2Jets_NuNu_ZpT_50To150_18', 'Z2Jets_NuNu_ZpT_150To250_18', 'Z2Jets_NuNu_ZpT_250To400_18', 'Z2Jets_NuNu_ZpT_400Toinf_18'])